In [1]:
import os
os.chdir(os.path.join(os.getcwd(), "../"))

In [13]:
from dotenv import load_dotenv
load_dotenv()
os.environ["MLFLOW_TRACKING_URI"]= os.getenv("DAGSHUB_MLFLOW_TRACKING_URI")
os.environ["MLFLOW_TRACKING_USERNAME"]= os.getenv("DAGSHUB_MLFLOW_USERNAME")
os.environ["MLFLOW_TRACKING_TOKEN"]= os.getenv("DAGSHUB_PAT")


In [5]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Stone",
            mlflow_uri="https://dagshub.com/ishansharma1320/kidney-disease-classification-mlflow-e2e.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="InceptionV3")
            else:
                mlflow.keras.log_model(self.model, "model")

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-09-15 19:22:29,239: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-15 19:22:29,244: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-15 19:22:29,245: INFO: common: created directory at: artifacts]
Found 2208 images belonging to 1 classes.
138/138 [==============================] - 85s 619ms/step - loss: 70.1002 - accuracy: 0.0562
[2025-09-15 19:23:55,797: INFO: common: json file saved at: scores.json]


2025/09/15 19:23:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/bx/q3zd25d93r7bb55tp6fnk8lm0000gn/T/tmpw0x_3pq4/model/data/model/assets
[2025-09-15 19:24:05,281: INFO: builder_impl: Assets written to: /var/folders/bx/q3zd25d93r7bb55tp6fnk8lm0000gn/T/tmpw0x_3pq4/model/data/model/assets]


2025/09/15 19:24:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'InceptionV3'.
2025/09/15 19:24:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: InceptionV3, version 1
Created version '1' of model 'InceptionV3'.
2025/09/15 19:24:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-shad-42 at: https://dagshub.com/ishansharma1320/kidney-disease-classification-mlflow-e2e.mlflow/#/experiments/0/runs/c3918879963d43ef83eebfe5e17ead31.
2025/09/15 19:24:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/ishansharma1320/kidney-disease-classification-mlflow-e2e.mlflow/#/experiments/0.
